In [75]:
import pandas as pd
import re
import os
import pickle
import numpy as np
import collections
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")


from add_lib import absolute_lib_path

[nltk_data] Downloading package punkt to
[nltk_data]     /home/m.vasilevskaia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/m.vasilevskaia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
def tag(sentence):
    words = tokenize(sentence)
    tagged_words = predictor.predict(words)
    return tagged_words

def tokenize(sentence):
    sentence = re.sub('\(', ',', sentence)
    sentence = re.sub('\)', '', sentence)
    sentence = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', sentence)
    words = sentence.split()
    return words

def preparse(sentence):
    tagged_words = tag(sentence)
    
    degree_re = re.compile('(.*)\|Degree=\w+(.*)')
    gender_re = re.compile('(.*)\|Gender=\w+(.*)')
    res = ''
    
    for word in tagged_words:
        word_tag = word.tag
        if 'Degree' in word_tag:
            word_tag = degree_re.sub(r"\1\2", word_tag)

        word_pos = 'pos=' + word.pos#.lower().capitalize()
        if 'PUNCT' in word_pos:
            word_pos = ''

        if 'ADJ' in word_pos:
            if 'Gender' in word_tag:
                word_tag = gender_re.sub(r"\1\2", word_tag)

        word_tags = [
                word.normal_form,
                word_pos,
                word_tag.replace('|', ' ').replace('_', ''),
            ]
        res += ' ' + ' '.join([tag for tag in word_tags if tag])

    res = res.strip().replace('  ', ' ')
    return res

def norm(sentence):
    tagged_words = tag(sentence)
    return ' '.join(word.normal_form for word in tagged_words)

In [85]:
norm('штатные расписания росприроднадзора и его территориальных органов')

'штатный расписание росприроднадзор и он территориальный орган'

In [3]:
# путь к распарсенным отчетам на Ubuntu
data_dir = '/home/common/regulation_data/parsed_reports/'

# main_df – таблица, куда сложены заполнения one-to-one
main_df = pd.read_csv(data_dir + 'main_df.csv')

# загрузка one-to-many таблиц
otm_tables_fnames = sorted([fn for fn in os.listdir(data_dir) if not fn.startswith('main')])
otm_tables = {fn[:-4]: pd.read_csv(data_dir + fn) for fn in otm_tables_fnames}
display(otm_tables.keys())

dict_keys(['0_business', '1_business', '2_business', 'cancel_duties', 'expenses', 'goals', 'group_changes', 'group_expenses', 'groups', 'kpi', 'neccessary_measures', 'new_functions', 'notification_info', 'public_discussion', 'risks'])

### Гайдлайны

Хорошее заполнение: ссылки на конкретные работы / справочники по годам

Отсутствие информации, непроведение оценки, указание закона, документа/ссылки.

In [61]:
def cleanup(text):
    text = str(text)
    text = text.lower()
    text = re.sub('[,\.]$', '', text)
    return text

def contains(text, patterns_list):
    for pattern in patterns_list:
        if pattern in text:
            return True

In [86]:
goals = otm_tables['goals']['Цели предлагаемого регулирования'].to_list()
sources = main_df['groups: Источники данных'].apply(cleanup).apply(norm).to_list()

In [89]:
from collections import Counter
src_counter = Counter(sources)

In [90]:
registers = ['реестр', 'перечень', 'регистр']

registers_dict = {x for x in src_counter.keys() if contains(x, registers)}
print(len(registers_dict), len(src_counter))

348 2009


In [91]:
src_counter_left = {k: v for k,v in src_counter.items() if k not in registers_dict}
sorted(src_counter.items(), key=lambda item: (-item[1], item[0]), reverse=True)

[('№ 481-фз от 27 декабрь 2019 год "о внесение изменение в отдельный законодательный акт российский федерация',
  1),
 ('экспертный оценка – сведение росфинмониторинг об организация и индивидуальный предприниматель , состоящий на учёт по состояние на 1 июнь 2016 года; сведение надзорный орган , в тот число указанный в реестр , роскомнадзор , открытый источник информация',
  1),
 ('экспертный оценка сотрудник минтранс россия', 1),
 ('экспертный оценка департамент государственный политика в область автомобильный и городской пассажирский транспорт минтранс россия , оценка российский союз промышленник и предприниматель , изложенный в позиция участник обсуждение проект постановление правительство российский федерация «о категория оснащаемый тахограф транспортный средство , осуществляющий регулярный перевозка пассажир , а также вид сообщение , в который осуществляться такой перевозка транспортный средство указанный категорий» . сведение о количество автомототранспортный средство и прицеп к о

In [118]:
statistics = ['данные', 'отчёт', 'отчётность', 'исследование', 'анализ', 'информация',
             'доклад', 'статистика']

text = '\n'.join(src_counter.keys())
tokens = nltk.word_tokenize(text)
nltk_text = nltk.Text(tokens)
nltk_text.concordance('данные', 200)

Displaying 25 of 331 matches:
аналитический информация росрыболовство , данные научный исследование информация уточняться фбу `` росавтотранс '' водный законодательство статис
кс российский федерация нет росалкогольрегулирование отсутствовать единый реестр лицензия г - по данные росприроднадзор ассоциация частный агентство занятость , ачазый данные разработчик данные о субъ
ый реестр лицензия г - по данные росприроднадзор ассоциация частный агентство занятость , ачазый данные разработчик данные о субъектный состав группа участник отношение сформулированный исходить из су
г - по данные росприроднадзор ассоциация частный агентство занятость , ачазый данные разработчик данные о субъектный состав группа участник отношение сформулированный исходить из существующий классифи
мов » форма федеральный статистический наблюдение №12 пу « сведение по похоронный обслуживанию » данные , предоставленный лицензирующий орган росгидромет изучение российский и международный принцип ре
я минстрой россия ин

### Правило для статистик

DATA_OWNERS = операторы связи | ниу | ассоциация | фонд | нбо | оператор

STAT := 'данные' | 'отчёт' | 'отчётность' | ...

GOOD_STAT := STAT + WHOSE + WHICH + YEAR

WHOSE := AUTH | DATA_OWNERS

WHICH := по + NP | о + NP | NP + родительный_падеж  

In [109]:
[x for x in src_counter.keys() if contains(x, statistics)]

['аналитический информация росрыболовство , данные научный исследование',
 'информация уточняться',
 'статистический отчётность ростехнадзор',
 'информация росрыболовство',
 'по данные росприроднадзор',
 'данные разработчик',
 'данные о субъектный состав группа участник отношение сформулированный исходить из существующий классификация участник перевозочный процесс , установленный законодательство российский федерация в область железнодорожный транспорт , а также исходить из существующий система взаимоотношение между он',
 'информация отсутствовать',
 'данные , предоставленный лицензирующий орган росгидромет',
 'изучение российский и международный принцип регулирование использование радиочастотный спектр , анализ обращение оператор связь , физический и юридический лиц-пользователь радиочастотный спектр',
 'информация о точный количество участник отношение отсутствовать',
 'статистический данные росавиация',
 'отчёт рослесхоз',
 'данные росстат , полученный из открытый источник',
 'внутр

### Пропуски

'nan'

\-

\---

### Отсутствие информации

(информация) 'отсутствует' во всех словоформах

нет

не имеется

In [17]:
for c in otm_tables['2_business'].columns:
    print('otm_tables[\'2_business\'][\'' + c + '\'],')

otm_tables['2_business']['id'],
otm_tables['2_business']['Прогноз количественной динамики структуры регулируемых субъектов по категориям при введении предлагаемого регулирования'],
otm_tables['2_business']['Микропредприятия'],
otm_tables['2_business']['Малыепредприятия'],
otm_tables['2_business']['Средниепредприятия'],


In [ ]:
# Сюда нужно добавить итого издержки для бизнеса, когда они переедут в main_df

numerical_cols = [
    otm_tables['groups']['Оценка количества участников отношений'],
    otm_tables['0_business']['Количественная (интервальная) оценка'],
    otm_tables['0_business']['Удельный вес(%)'],
    otm_tables['1_business']['Содержательные издержки: Единовременные'],
    otm_tables['1_business']['Содержательные издержки: Периодические'],
    otm_tables['1_business']['Информационные издержки: Единовременные'],
    otm_tables['1_business']['Информационные издержки: Периодические'],
    otm_tables['1_business']['Преимущества и (или) иные выгоды: Единовременные'],
    otm_tables['1_business']['Преимущества и (или) иные выгоды: Периодические'],
    otm_tables['2_business']['Микропредприятия'],
    otm_tables['2_business']['Малыепредприятия'],
    otm_tables['2_business']['Средниепредприятия'],
    
    
]